In [27]:
import numpy as np
import scipy
import scipy.linalg

# 3.30

To preserve the norm, we must have $\alpha=\pm\|a\|_{2}$, and the sign should be chosen to avoid cancellation (i.e., $\alpha=-\operatorname{sign}\left(a_{1}\right)\|a\|_{2}$ ).

# 3.33

## (a)
需要$2n^2、3$的额外计算量。
## (b)
需要$2n^2(m-n/3)$的额外计算量。



# 3.44

SVD分解，schmidt正交化

# 3.45
(a)>(b)

# 3.3

$$A =\left[\begin{array}{ll}
1 & e \\
2 & e^2\\
3 & e^3
\end{array}\right]$$

$$x = (A^TA)^{-1}A^Tb=[1.59419483 0.00876201]$$

In [28]:
A = np.array([[1,np.exp(1)],[2,np.exp(2)],[3,np.exp(3)]])
b = np.array([2,3,5])
x = np.linalg.inv(A.T@A)@A.T@b
print(x)

[1.59419483 0.00876201]


# 3.14

$$
H=I-2 \frac{\boldsymbol{v} v^{T}}{\boldsymbol{v}^{T} \boldsymbol{v}}
$$

$$
H^T = I-2 \frac{\boldsymbol{v} v^{T}}{\boldsymbol{v}^{T} \boldsymbol{v}} = H
$$

$$
H^T H = (I-2 \frac{\boldsymbol{v} v^{T}}{\boldsymbol{v}^{T} \boldsymbol{v}})(I-2 \frac{\boldsymbol{v} v^{T}}{\boldsymbol{v}^{T} \boldsymbol{v}}) = I - 4 \frac{\boldsymbol{v} v^{T}}{\boldsymbol{v}^{T} \boldsymbol{v}} + 4 \frac{\boldsymbol{v} (v^{T}\boldsymbol{v}) v^{T}}{(\boldsymbol{v}^{T} \boldsymbol{v})^2}=I
$$

H对称正定。

In [29]:
def Q_i(Q_min, i, j, k):
    if i < k or j < k:
        return float(i == j)
    else:
        return Q_min[i-k][j-k]

def householder(A):
    """Performs a Householder Reflections based QR Decomposition of the                                               
    matrix A. The function returns Q, an orthogonal matrix and R, an                                                  
    upper triangular matrix such that A = QR."""
    n = A.shape[0]

    # Set R equal to A, and create Q as a zero matrix of the same size
    R = A.copy()
    Q = np.zeros((n,n))

    # The Householder procedure
    for k in range(n-1):
        I = np.eye(n)
        x = R[k:,k]
        e = I[k:,k]
        alpha = -np.sign(x[0]) * np.linalg.norm(x)

        # Using anonymous functions, we create u and v
        u = np.array(list(map(lambda p,q: p + alpha * q, x, e)))
        print(u)
        norm_u = np.linalg.norm(u)
        v = np.array(list(map(lambda p: p/norm_u, u)))

        # Create the Q minor matrix
        Q_min = np.array([ [float(i==j) - 2.0 * v[i] * v[j] for i in range(n-k)] for j in range(n-k) ])

        # "Pad out" the Q minor matrix with elements from the identity
        Q_t = np.array([[ Q_i(Q_min,i,j,k) for i in range(n)] for j in range(n)])

        if k == 0:
            Q = Q_t
            R = Q_t@A
        else:
            Q = Q_t@Q
            R = Q_t@R

    # Since Q is defined as the product of transposes of Q_t,
    # we need to take the transpose upon returning it
    return Q.T, R

In [30]:
A = np.array([[12, -51, 4], [6, 167, -68], [-4, 24, -41]])
Q, R = householder(A)

[-2.  6. -4.]
[126. 168.]


In [31]:
Q,R

(array([[ 0.85714286,  0.39428571, -0.33142857],
        [ 0.42857143, -0.90285714,  0.03428571],
        [-0.28571429, -0.17142857, -0.94285714]]),
 array([[ 1.40000000e+01,  2.10000000e+01, -1.40000000e+01],
        [-5.50670620e-16, -1.75000000e+02,  7.00000000e+01],
        [ 3.01980663e-16, -2.84217094e-14,  3.50000000e+01]]))

In [32]:
scipy.linalg.qr(A)

(array([[-0.85714286,  0.39428571,  0.33142857],
        [-0.42857143, -0.90285714, -0.03428571],
        [ 0.28571429, -0.17142857,  0.94285714]]),
 array([[ -14.,  -21.,   14.],
        [   0., -175.,   70.],
        [   0.,    0.,  -35.]]))